## Wordbank country indicator analysis
### import libraries

In [3]:
import pandas as pd
import requests

## Path to excel file

In [5]:
path='C:\\Users\\yan_e\\Desktop\\JupyterLab\\Data-sheet-STUDENTS.xlsx'

### excelTab get the excel file header is LINE3 of excel (Country, Date, HF Index, Heritage...)

In [7]:
excelTab = pd.read_excel(path,header=2,index_col=0)

In [9]:
##excelTab['Country (according to CN Gov):'] ##Show list of countries

## Getting isocode3 + exceptions (India, Thialand, UAE, paranthesis city)

In [11]:
restCountryURL = 'https://restcountries.eu/rest/v2/name/'
reqAnswer = ''
isoCodes = {}

In [12]:
for index , name in excelTab['Country (according to CN Gov):'].items():
    if name in isoCodes:
        continue
    reducedName= name.split('(')[0]
    reducedName=reducedName.strip()
    if reducedName == 'Thailiand':
        reducedName = 'thailand'
    if reducedName == 'The United Arab Emirates':
        reducedName = 'United Arab Emirates'
    ##if name[-1:]==' ':
    ##    name = name[:-1]
    req = requests.get(restCountryURL+reducedName)
    if req.status_code ==200:
        reqAnswer = req.json()
        countryCode = reqAnswer[0]['alpha3Code']
        if countryCode =='IOT':
            countryCode = reqAnswer[1]['alpha3Code']
        isoCodes[name] = countryCode
    else:
        print('[ERROR]', req.status_code, req.text , name)

In [14]:
excelTab['isocode'] = excelTab['Country (according to CN Gov):'].apply(lambda x: isoCodes[x]) ##Apply isocode to excel

In [41]:
indicatorDone={}
valueData = {}
indicators = ['FB.AST.NPER.ZS','GC.TAX.IMPT.ZS','IS.RRS.GOOD.MT.K6','NE.CON.PRVT.PP.CD','GB.XPD.RSDV.GD.ZS','SP.POP.SCIE.RD.P6','BX.KLT.DINV.WD.GD.ZS','SH.XPD.CHEX.PP.CD','GB.XPD.RSDV.GD.ZS','IP.TMK.RSCT','IP.PAT.RESD']

In [43]:
print(indicatorDone)
print('###############################')
print(valueData)


{'FB.AST.NPER.ZS': 'FB.AST.NPER.ZS', 'GC.TAX.IMPT.ZS': 'GC.TAX.IMPT.ZS', 'IS.RRS.GOOD.MT.K6': 'IS.RRS.GOOD.MT.K6', 'NE.CON.PRVT.PP.CD': 'NE.CON.PRVT.PP.CD', 'GB.XPD.RSDV.GD.ZS': 'GB.XPD.RSDV.GD.ZS'}
###############################
{'AFG': 0, 'ALB': 0, 'ARM': 0, 'AZE': 0, 'BHR': 368.90235, 'BGD': 0, 'BLR': 0, 'BTN': 0, 'BIH': 463.89852, 'BRN': 0, 'BGR': 2243.70938, 'KHM': 30.37447, 'CHN': 1205.68248, 'HRV': 1793.14617, 'CZE': 3518.81882, 'TLS': 0, 'EGY': 669.39201, 'EST': 3305.28892, 'FRA': 4307.22237, 'GEO': 1336.56272, 'DEU': 4893.15272, 'HUN': 2645.6738, 'IND': 216.18207, 'IDN': 89.19929, 'IRN': 671.02399, 'IRQ': 103.92502, 'ISR': 8250.47418, 'JOR': 601.0913, 'KAZ': 687.63518, 'KWT': 491.81342, 'KGZ': 0, 'LAO': 0, 'LVA': 1599.56966, 'LBN': 0, 'LTU': 2931.66094, 'MKD': 728.95025, 'MYS': 2273.98521, 'MDV': 0, 'MDA': 723.88211, 'MNG': 0, 'MNE': 832.99638, 'MMR': 15.28931, 'NPL': 0, 'OMN': 243.96162, 'PAK': 293.6469, 'PHL': 187.65898, 'POL': 2158.46105, 'QAT': 603.79063, 'ROU': 912.42412

In [44]:
for indic in indicators:
    if indic in indicatorDone:
        continue
    for index , name in excelTab['isocode'].items():
        req = requests.get('http://api.worldbank.org/v2/country/'+excelTab['isocode'][index]+'/indicator/'+indic+'?format=json')
        reqAnswer = req.json()
        if req.status_code == 200:
            indexJ=1
            done = 0
            while not done : 
                if indexJ<len(reqAnswer[1]):
                    if reqAnswer[1][indexJ]['value']!=None:
                        done=1
                        valueData[name] = reqAnswer[1][indexJ]['value']
                    if indexJ==10:    
                        valueData[name] = 0
                        break
                else:
                    valueData[name]=0
                    break
                indexJ=indexJ+1
        else:
            print('Answer not 200 : ')
            print (req.status_code)
    if (indic == 'FB.AST.NPER.ZS'):
        excelURL = 'WB IMF Non-performing loans to total gross loans ratio (%)'
    elif (indic == 'GC.TAX.IMPT.ZS' ):
        excelURL = 'IMF WB \nCustoms and other import duties (% of tax revenue) 2016'
    elif(indic == 'IS.RRS.GOOD.MT.K6'):
        excelURL = 'UIC Rail freight (mn ton x km travelled)'
        excelTab[excelURL] = excelTab['isocode'].apply(lambda x: valueData[x])
        excelURL = 'ICAO air freight (mn ton x km travelled)'
    elif (indic =='NE.CON.PRVT.PP.CD'):
        excelURL = 'WB Household final consumption expenditure, PPP bn USD'
    elif (indic == 'GB.XPD.RSDV.GD.ZS'):
        excelURL = 'WB UNESCO R&D Expenditure % of GDP'
    elif (indic =='SP.POP.SCIE.RD.P6'):
        excelURL = 'WB UNESCO Researchers p. mn inhabitants'
    elif (indic =='BX.KLT.DINV.WD.GD.ZS'):
        excelURL = 'IFDI Net Inflows (% of GDP)'
    elif (indic == 'SH.XPD.CHEX.PP.CD'):
        excelURL = 'WHO Current Health Expenditure p.c., PPP USD'
    elif (indic == 'GB.XPD.RSDV.GD.ZS'):
        excelURL = 'WB Research Expenditure % of GDP'
    elif (indic == 'IP.TMK.RSCT'):
        excelURL = 'WIPO Trademark applications'
    elif (indic == 'IP.PAT.RESD'):
            excelURL = 'WIPO Patent Applications'  
    indicatorDone[indic] = indic
    excelTab[excelURL] = excelTab['isocode'].apply(lambda x: valueData[x])
    print ('Line ' +excelURL+' mise a jour')
    if indic=='SP.POP.SCIE.RD.P6':
        break

Line WB UNESCO Researchers p. mn inhabitants mise a jour


In [39]:
excelTab[['isocode','WB IMF Non-performing loans to total gross loans ratio (%)','IMF WB \nCustoms and other import duties (% of tax revenue) 2016','UIC Rail freight (mn ton x km travelled)','ICAO air freight (mn ton x km travelled)','WB Household final consumption expenditure, PPP bn USD','WB UNESCO R&D Expenditure % of GDP','WB UNESCO Researchers p. mn inhabitants', 'IFDI Net Inflows (% of GDP)', 'WHO Current Health Expenditure p.c., PPP USD', 'WB Research Expenditure % of GDP','WIPO Trademark applications','WIPO Patent Applications' ]]

,isocode,WB IMF Non-performing loans to total gross loans ratio (%),IMF WB Customs and other import duties (% of tax revenue) 2016,UIC Rail freight (mn ton x km travelled),ICAO air freight (mn ton x km travelled),"WB Household final consumption expenditure, PPP bn USD",WB UNESCO R&D Expenditure % of GDP,WB UNESCO Researchers p. mn inhabitants,IFDI Net Inflows (% of GDP),"WHO Current Health Expenditure p.c., PPP USD",WB Research Expenditure % of GDP,WIPO Trademark applications,WIPO Patent Applications
1,AFG,12.201830,35.203141,0.000000e+00,0.000000e+00,5.214309e+10,0.00000,NaN,NaN,NaN,NaN,NaN,NaN
2,ALB,13.232533,2.216459,4.600000e+01,4.600000e+01,2.386004e+10,0.00000,NaN,NaN,NaN,NaN,NaN,NaN
3,ARM,5.432318,6.275395,6.894600e+02,6.894600e+02,2.288547e+10,0.22770,NaN,NaN,NaN,NaN,NaN,NaN
4,AZE,0.000000,7.571847,4.633000e+03,4.633000e+03,9.668125e+10,0.18521,NaN,NaN,NaN,NaN,NaN,NaN
5,BHR,0.000000,0.000000,0.000000e+00,0.000000e+00,2.447994e+10,0.10116,NaN,NaN,NaN,NaN,NaN,NaN
6,BGD,8.397908,28.923008,7.100000e+02,7.100000e+02,4.111268e+11,0.00000,NaN,NaN,NaN,NaN,NaN,NaN
7,BLR,12.852194,8.200619,4.853800e+04,4.853800e+04,1.143158e+11,0.58716,NaN,NaN,NaN,NaN,NaN,NaN
8,BTN,8.417308,2.840501,0.000000e+00,0.000000e+00,3.833999e+09,0.00000,NaN,NaN,NaN,NaN,NaN,NaN
9,BIH,10.046275,0.000155,1.114327e+03,1.114327e+03,3.126192e+10,0.19951,NaN,NaN,NaN,NaN,NaN,NaN
10,BRN,3.538399,0.000000,0.000000e+00,0.000000e+00,4.427808e+09,0.00000,NaN,NaN,NaN,NaN,NaN,NaN
